In [ ]:
%cd /local/home/ahemf/mygit/facebook-hateful-memes/notebooks/
%pwd
!pip uninstall -y maskrcnn_benchmark
!rm -rf vqa-maskrcnn-benchmark
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark.git
# !sed -i '/from maskrcnn_benchmark import _C/c\from ._utils import _C' maskrcnn_benchmark/layers/nms.py
!cat maskrcnn_benchmark/layers/nms.py
%cd vqa-maskrcnn-benchmark

# %sed -i '/from maskrcnn_benchmark import _C/c\from ._utils import _C' layers/nms.py
# %cat layers/nms.py
%pwd
!python setup.py build
!python setup.py develop
!wget -O detectron_model.pth wget https://dl.fbaipublicfiles.com/vilbert-multi-task/detectron_model.pth
!wget -O detectron_model.yaml wget https://dl.fbaipublicfiles.com/vilbert-multi-task/detectron_config.yaml

import sys
import os
DIR = os.getcwd()
sys.path.append(f'{DIR}/vqa-maskrcnn-benchmark')


In [ ]:
# %pip install git+https://github.com/faizanahemad/ImageCaptioning.pytorch.git
%pip install gdown
!gdown --id 1VmUzgu0qlmCMqM1ajoOZxOXP3hiC_qlL
!gdown --id 1zQe00W02veVYq-hdq5WsPOS3OPkNdq79
!pip install yacs



%cd ..
%mkdir model_data
!pip uninstall -y captioning
%pip install git+https://github.com/faizanahemad/ImageCaptioning.pytorch.git


In [1]:
import pandas as pd
import numpy as np

from PIL import Image
from IPython.display import display, HTML, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys
import os
DIR = os.getcwd()
import torch
import torchvision
from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut
from PIL import Image
from facebook_hateful_memes_detector.utils import get_image_info_fn, set_device, get_device


def print_code(func):
    import inspect
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import TerminalFormatter

    code = "".join(inspect.getsourcelines(func)[0])
    print(highlight(code, PythonLexer(), TerminalFormatter()))

set_device('cpu')


In [2]:
from facebook_hateful_memes_detector.models.external.detr import DETR
detr = DETR(get_device())


ModuleNotFoundError: No module named 'util'

In [ ]:
!wget https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/images/input.jpg


In [2]:
im_transform = torchvision.transforms.RandomChoice([torchvision.transforms.Grayscale(num_output_channels=3), torchvision.transforms.RandomHorizontalFlip(p=1.0), 
                                    DefinedRotation(15), QuadrantCut(), lambda x: x])

In [ ]:
im_transform(Image.open("input.jpg"))

In [ ]:
torchvision.transforms.RandomPerspective(p=1.0)(Image.open("input.jpg"))

In [4]:
fns = get_image_info_fn(enable_encoder_feats=True, enable_image_captions=True)


Config '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/utils/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [9]:
res, info = fns["feature_extractor"](Image.open("../data/img/08291.png"))
res[:2, :8]
info["boxes"][:4]
info["objects"]


tensor([[ 0.0000,  4.0681,  0.0000,  0.0000,  6.1578,  9.3466,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, 19.3470,  0.0000,  0.0000,  0.0000]])

array([[274.356  , 245.17194, 482.75903, 478.62283],
       [464.95288, 242.99124, 814.4191 , 507.37842],
       [446.2343 ,  72.23443, 833.1075 , 569.58746],
       [245.07552,  51.55629, 429.297  , 233.34947]], dtype=float32)

array([ 52, 227, 120,  52, 227,  52,  52,   0, 365, 120, 227,   0,   0,
         0, 227,   0,   0,   0,   0,   0,   0,   0,   0, 120,   0,   0,
         0,   0,   0,   0,   0, 510,   0,   0, 227,   0,   0, 227,   0,
         0,   0, 227,   0,   0, 227,   0,   0, 227,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0, 227,   0,   0,   0,   0,   0,
         0,   0,   0,  52,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 227,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,  52,   0,   0,   0,   0])

In [8]:
res, info = fns["get_img_details"](Image.open("../data/img/08291.png"))
res[:2, :8]
info["boxes"][:4]
info["objects"]


tensor([[0.0000, 3.1754, 0.0000, 0.0000, 2.3690, 8.6731, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 4.1627, 2.1554, 0.0000, 0.0000]])

array([[271.8734 , 242.58258, 477.367  , 483.0695 ],
       [515.4416 , 480.6227 , 707.6861 , 548.3137 ],
       [207.75957, 285.3643 , 452.54108, 548.3137 ],
       [610.19604,  86.59137, 752.1002 , 217.54742]], dtype=float32)

array([  48,   52,  120,  120,  120,  662,  227,  224,  120,   52,   52,
        227,  120, 1322,   52,   52,   91,  441,   52,  227,  373,  243,
        373,  227,  227,  243,  771,   99,    0,  243,  227,  662,    0,
          0,  120,    0,  585,    0,  192,   52,    0,  243,    0,  227,
          0,  227,  208,    0,    0,    0,    0,  441,    0,   99,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,  771,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   91,
        510,    0,    0,    0,  224,    0,    0,   91,    0,    0,  243])

In [11]:
fns["get_image_captions"]("../data/img/08291.png")

['two young boys are smiling for the camera',
 'two young boys are smiling at the camera',
 'two young boys standing next to each other',
 'two little boys standing next to each other',
 'two young boys standing next to each other with food']

In [ ]:
im = im_transform(Image.open("input.jpg"))
im

In [20]:
instances, roi_features = fns["get_lxmert_details"](im)
instances.pred_boxes.tensor # boxes
roi_features # feats
# (feats, boxes)

tensor([[1.4383e+02, 2.1078e+02, 4.6714e+02, 4.7354e+02],
        [1.7428e+02, 3.1377e+02, 5.1336e+02, 4.8000e+02],
        [2.8904e+02, 2.0981e+02, 5.1901e+02, 4.8000e+02],
        [2.1644e+02, 1.8520e+02, 6.2635e+02, 4.6370e+02],
        [3.3843e-02, 0.0000e+00, 4.1693e+01, 3.7946e+02],
        [3.1173e+02, 1.5900e+02, 3.5625e+02, 1.9688e+02],
        [2.2946e+02, 2.2712e+02, 3.0838e+02, 2.9746e+02],
        [4.6132e+00, 2.6840e+02, 1.9707e+02, 3.8635e+02],
        [2.4226e+00, 3.0217e+02, 2.7723e+02, 4.0444e+02],
        [4.5615e+02, 7.9694e+00, 6.1806e+02, 4.5733e+02],
        [2.9042e+02, 1.7090e+02, 3.9177e+02, 4.1011e+02],
        [1.2967e+00, 1.8283e+00, 1.8816e+02, 3.6328e+02],
        [5.0785e+02, 6.4634e+01, 5.8013e+02, 4.4548e+02],
        [3.3803e+02, 0.0000e+00, 4.3406e+02, 2.7469e+02],
        [1.9479e+02, 1.2730e+02, 4.9682e+02, 4.1328e+02],
        [5.9692e+01, 2.7249e+02, 3.9423e+02, 4.8000e+02],
        [1.1351e+02, 3.3084e+02, 6.4000e+02, 4.7781e+02],
        [1.481

tensor([[0.0000, 0.0099, 0.0090,  ..., 6.5851, 0.0370, 7.3452],
        [0.0000, 0.0963, 0.4266,  ..., 2.0185, 0.0253, 5.7607],
        [0.0000, 0.0968, 0.1323,  ..., 5.3939, 0.0000, 7.1907],
        ...,
        [0.0000, 0.5091, 0.3767,  ..., 5.4250, 0.1418, 0.0369],
        [0.2100, 0.2867, 0.2244,  ..., 0.0330, 2.4388, 0.1414],
        [0.0000, 0.6733, 0.3991,  ..., 1.3741, 0.6837, 0.0531]],
       device='cuda:0')

In [7]:
fns["get_encoder_feats"]("../data/img/08291.png")


tensor([[[ 0.5593, -0.0353, -0.8734,  ..., -0.0888, -0.3785, -0.5709],
         [-0.0509,  0.1510, -0.1381,  ...,  0.3084,  0.2411,  0.0265],
         [ 0.3100,  0.0862, -0.9152,  ...,  0.1054, -0.4900, -0.5777],
         ...,
         [-0.3711,  0.1200,  0.0026,  ...,  0.0058,  0.0990,  0.0793],
         [ 0.1025, -0.6312, -0.0860,  ..., -0.4207, -0.8504, -0.5901],
         [-0.3204,  0.0824, -0.0489,  ..., -0.0487,  0.0735,  0.0672]]])